# Decision Tree with ID3

In [1]:
import numpy as np
from scipy.stats import entropy
from collections import defaultdict, Counter
import pandas as pd

In [2]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

In [3]:
df = pd.DataFrame.from_dict(inputs)
for feature in ['level', 'lang', 'tweets', 'phd']:
    df[feature] = df.apply(lambda row: row[0][feature],axis=1)
df = df.rename(columns={1:'label'}).drop(0,axis=1)

In [4]:
samples = df.copy()

In [5]:
def compute_split_entropy(candidate_df, attribute):
    
    entropy_list = list()
    
    total_count = candidate_df['sample_count'].values.sum()
    
    for i, grp in candidate_df.groupby(attribute,as_index=False):
        
        entropy_list.append(grp['sample_count'].values.sum()/total_count*entropy(grp.probability.values))
        
    return sum(entropy_list)

In [6]:
def add_probability_for_attribute(samples, attribute):
    
    df = samples.copy()
    
    df['sample_count'] = 1
    
    df_attribute_label = df.groupby([attribute, 'label'], as_index=False).agg({'sample_count':'count'})
            
    df_attribute = df.groupby([attribute], as_index=False).agg({'sample_count':'count'}).rename(
        columns={'sample_count':'total'})
    
    df = pd.merge(df_attribute_label, df_attribute, on=[attribute], how='left')
    df['probability'] = df['sample_count']/df['total']
    
    return df.drop('total',axis=1)

In [7]:
def get_best_attribute(samples):
        
    number_of_samples, _ = samples.shape
    candidate_attributes = samples.drop('label', axis=1).columns

    attribute_score = dict()

    for attribute in candidate_attributes:

        df = add_probability_for_attribute(samples, attribute)
        attribute_score[attribute] = compute_split_entropy(df, attribute)

    return min(attribute_score, key=lambda k: attribute_score[k]) 

In [8]:
def split_samples(samples, attribute):
    return {attribute_value : samples[samples[attribute] == attribute_value].drop(attribute, axis=1) 
            for attribute_value in samples[attribute].unique()}

In [9]:
class TreeNode():
    
    def __init__(self, samples,is_root=False):
        
        self.is_root = is_root
        self.split_attribute = None
        self.child_nodes = dict()
        self.samples = samples
        self.classifiction = None
        self.make_split()
    
    def make_split(self):
    
        if self.samples.label.nunique() == 1:

            self.classifiction = self.samples.label.unique()[0]
            
        elif len(self.samples.columns) == 2:
            
            attribute = self.samples.columns.drop('label')[0]
            df = self.samples.groupby('label', as_index=False).agg({attribute:'count'})
            print(df)
            self.classifiction = df[df[attribute]==df[attribute].max()].label.values[0]
            
        else:
            self.split_attribute = get_best_attribute(samples=self.samples)

            new_samples = split_samples(self.samples, self.split_attribute)
            
            for attribute_value, new_sample in new_samples.items():
                self.child_nodes[attribute_value] = TreeNode(new_sample)
            
        

In [10]:
tn = TreeNode(samples,is_root=True)

In [11]:
tn.child_nodes.keys()

dict_keys(['Senior', 'Mid', 'Junior'])

In [12]:
tn.child_nodes['Junior'].samples

,label,lang,tweets,phd
3,True,Python,no,no
4,True,R,yes,no
5,False,R,yes,yes
9,True,Python,yes,no
13,False,Python,no,yes


In [13]:
tn.child_nodes['Junior'].split_attribute

'phd'

In [14]:
tn.child_nodes['Junior'].child_nodes['no'].classifiction

True

In [15]:
tn.child_nodes['Junior'].child_nodes['no'].child_nodes

{}

In [16]:
tn.child_nodes['Junior'].classifiction

In [17]:
def classify(sample,tree):
    if tree.classifiction != None:
        return tree.classifiction
    else:
        return classify(sample,tree.child_nodes[sample[tree.split_attribute]])

In [18]:
samples

,label,level,lang,tweets,phd
0,False,Senior,Java,no,no
1,False,Senior,Java,no,yes
2,True,Mid,Python,no,no
3,True,Junior,Python,no,no
4,True,Junior,R,yes,no
5,False,Junior,R,yes,yes
6,True,Mid,R,yes,yes
7,False,Senior,Python,no,no
8,True,Senior,R,yes,no
9,True,Junior,Python,yes,no


In [19]:
for i in range(14):
    print(classify(samples.iloc[i].to_dict(),tn))

False
False
True
True
True
False
True
False
True
True
True
True
True
False
